In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

#Using webdriver to go to Google Scholar
driver = webdriver.Chrome()
driver.get("https://scholar.google.com/schhp?hl=en&as_sdt=0,48")

#Finding search bar, inputting parameter and hitting enter
search = driver.find_element(By.NAME, 'q')
search.send_keys("bats and kittens")
search.send_keys(Keys.RETURN)

#getting current url for BS webscrapping, setting up lists for DF
url = driver.current_url
titles = []
links = []
authors = []
description = []

#Scraping title, link, authors, and description, adding to lists then adding lists to DF
response=requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')
    

df = pd.DataFrame(list(zip(titles, links, authors, description)), columns = ["Titles", "Links", "Authors", "Description"])

#Function to check if there is a next button on the page
def check_next_exists(driver, xpath):
    try:
        driver.find_element(By.XPATH,xpath)
    except NoSuchElementException:
        return False
    return True

more_pages = True

#While there is a next button on the webpage, scrap the webpage and add to dataframe, stop where there is no next button
while more_pages == True:
    for item in soup.select('[data-lid]'):
        t = item.select('h3')[0].get_text()
        titles.append(t)
        l = item.select('a')[0]['href']
        links.append(l)
        a = item.select('.gs_a')[0].get_text()
        authors.append(a)
        d = item.select('.gs_rs')[0].get_text()
        description.append(d)
    next_page = driver.find_element(By.XPATH, '//*[@id="gs_n"]/center/table/tbody/tr/td[12]/a/b')
    next_page.click()
    more_pages = check_next_exists(driver, "//*[@id='gs_n']/center/table/tbody/tr/td[12]/a/b")